<a href="https://colab.research.google.com/github/SilvanusTurkson/Face-Reg/blob/main/On_God1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
# Install Dependencies (Colab Only)
!pip install -q -U deepface streamlit tensorflow keras numpy opencv-python-headless pandas Pillow
!apt-get install -y libgl1-mesa-glx libglib2.0-0

# Import Libraries
import os
import pandas as pd
import numpy as np
from deepface import DeepFace
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Configuration
CONFIG = {
    "known_faces_dir": "/content/drive/MyDrive/known_faces",
    "unknown_faces_dir": "/content/drive/MyDrive/unknown_faces",
    "database_path": "/content/drive/MyDrive/face_embeddings.csv",
    "model_name": "Facenet",  # Change to "Facenet" if "ArcFace" is not available
    "threshold": 0.55,
    "image_extensions": ('.png', '.jpg', '.jpeg'),
    "detector": "retinaface"
}

# Helper Functions
def is_image_file(filename):
    return filename.lower().endswith(CONFIG['image_extensions'])

def clean_directory_listing(path):
    return [f for f in os.listdir(path) if is_image_file(f)]

# Extract Face Embeddings
def extract_embeddings():
    data = []
    print("⏳ Extracting face embeddings...")

    for person_name in os.listdir(CONFIG['known_faces_dir']):
        person_path = os.path.join(CONFIG['known_faces_dir'], person_name)

        if not os.path.isdir(person_path):
            continue

        for img_name in clean_directory_listing(person_path):
            img_path = os.path.join(person_path, img_name)
            print(f"Processing image: {img_path}")

            try:
                embedding = DeepFace.represent(
                    img_path=img_path,
                    model_name=CONFIG['model_name'],
                    detector_backend=CONFIG['detector'],
                    enforce_detection=True
                )[0]['embedding']

                data.append({
                    "name": person_name,
                    "embedding": np.array(embedding).tolist(),
                    "image_path": img_path
                })
                print(f"✅ Processed: {person_name}/{img_name}")

            except Exception as e:
                print(f"❌ Failed on {img_path}: {str(e)}")
                continue

    if data:
        pd.DataFrame(data).to_csv(CONFIG['database_path'], index=False)
        print(f"💾 Saved {len(data)} embeddings to {CONFIG['database_path']}")
    else:
        print("❌ No valid faces found!")

# Recognize Faces
def recognize_faces():
    try:
        df = pd.read_csv(CONFIG['database_path'])
        df['embedding'] = df['embedding'].apply(eval)  # Convert string to list
    except Exception as e:
        print(f"❌ Database error: {e}")
        return

    print("\n🔍 Starting recognition...")

    for img_name in clean_directory_listing(CONFIG['unknown_faces_dir']):
        img_path = os.path.join(CONFIG['unknown_faces_dir'], img_name)

        try:
            query_embedding = DeepFace.represent(
                img_path=img_path,
                model_name=CONFIG['model_name'],
                detector_backend=CONFIG['detector'],
                enforce_detection=True
            )[0]['embedding']
            query_embedding = np.array(query_embedding)

            similarities = [
                np.dot(query_embedding, np.array(row['embedding'])) /
                (np.linalg.norm(query_embedding) * np.linalg.norm(np.array(row['embedding'])))
                for _, row in df.iterrows()
            ]

            best_match_idx = np.argmax(similarities)
            best_similarity = similarities[best_match_idx]
            match_name = df.iloc[best_match_idx]['name']

            print(f"\n📸 Processing: {img_name}")
            print(f"🔎 Best Match: {match_name}")
            print(f"📏 Similarity: {best_similarity:.4f} (Threshold: {CONFIG['threshold']})")

            if best_similarity > CONFIG['threshold']:
                print(f"✅ VERIFIED: {match_name}")
            else:
                print("❌ UNKNOWN FACE")

        except Exception as e:
            print(f"⚠️ Error processing {img_name}: {str(e)}")

# Execute
print("⭐ Face Recognition System ⭐")
extract_embeddings()
recognize_faces()

# Summary
print("\n🔬 Summary")
print("Known faces:", os.listdir(CONFIG['known_faces_dir']))
print("Tested unknown faces:", clean_directory_listing(CONFIG['unknown_faces_dir']))


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.4).
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Mounted at /content/drive
⭐ Face Recognition System ⭐
⏳ Extracting face embeddings...
Processing image: /content/drive/MyDrive/known_faces/Swidess/12.jpg
✅ Processed: Swidess/12.jpg
Processing image: /content/drive/MyDrive/known_faces/Silvanus/5.jpg
✅ Processed: Silvanus/5.jpg
Processing image: /content/drive/MyDrive/known_faces/Trump/trump 3.jpeg
✅ Processed: Trump/trump 3.jpeg
Processing image: /content/drive/MyDrive/known_faces/Steve/steve1.jpg
✅ Processed: Steve/steve1.jpg
💾 Saved 4 embeddings to /content/drive/MyDrive/face_embeddings.csv

🔍 Starting recognition...

📸 Processing: trump 2.jpeg
🔎 Best Match: Trump
📏 Similarity: 0.5990 (Threshold: 0.55)
✅ VERIFIED: Trump

📸 Processing: steve2

In [81]:
df = pd.read_csv(DATABASE_PATH)
print(df.head())

       name                                          embedding  \
0   Swidess  [-0.5835148096084595, 0.23760107159614563, 0.2...   
1  Silvanus  [-0.19509676098823547, -1.0056116580963135, -1...   
2     Trump  [-0.6694222688674927, -0.03984030708670616, -0...   
3     Steve  [0.03908134624361992, -0.8783297538757324, 1.2...   

                                          image_path  
0  /content/drive/MyDrive/known_faces/Swidess/12.jpg  
1  /content/drive/MyDrive/known_faces/Silvanus/5.jpg  
2  /content/drive/MyDrive/known_faces/Trump/trump...  
3  /content/drive/MyDrive/known_faces/Steve/steve...  
